# Ugesco : enrich and desambiguate a sample of the Samnang-Hans' JSON

In [1]:
from ugesco import *

D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, fa

In [2]:
#test sur un Json contenant 20 éléments pris au hasard
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\jsondata_ugesco.json", "r", encoding="utf8") as file:
    json = json.load(file)
    
sample = random.sample(json, k=20)


In [3]:
data = pd.DataFrame([flatten(line) for line in sample])

In [4]:
data.head()

,beeldid,imageclassification_class1,imageclassification_class2,imageclassification_class3,imageclassification_class4,imageclassification_class5,imageclassification_prob1,imageclassification_prob2,imageclassification_prob3,imageclassification_prob4,...,spatial_key_1,spatial_key_2,spatial_value_0,spatial_value_1,spatial_value_2,temporal,temporal_key_0,temporal_key_1,temporal_value_0,temporal_value_1
0,00035678,street,plaza,promenade,downtown,canal/urban,0.378201,0.124322,0.120434,0.076608,...,LOC_BUILDING,NaN,Bruxelles la nuit,la Bourse,NaN,NaN,NaN,NaN,NaN,NaN
1,00039768,street,alley,arcade,courtyard,residential_neighborhood,0.336949,0.213263,0.091822,0.046664,...,NaN,NaN,Vieux Bruxelles,NaN,NaN,NaN,TIMEX_MISC,TIMEX_DATE,L' Exposition internationale et universelle de...,27 avril
2,00039607,street,bazaar/outdoor,medina,plaza,fire_station,0.276882,0.265026,0.074721,0.039008,...,NaN,NaN,Laeken,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00005577,plaza,street,crosswalk,promenade,campus,0.474215,0.214687,0.122037,0.120583,...,LOC_SPACES,LOC_PLACE,Avenue Louise,Bois de la Cambre,Bruxelles,NaN,NaN,NaN,NaN,NaN
4,00029855,street,medina,construction_site,alley,crosswalk,0.650091,0.105397,0.043445,0.031602,...,LOC_PLACE,NaN,Verviers,Verviers,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# exclude image classifications and temporal values for the moment
data = data[data.columns.drop(list(data.filter(regex='imageclassification|temporal')))]

In [6]:
# reshape the dataframe verticaly
data = data.set_index('beeldid')
data.columns = data.columns.str.split('_', expand=True)

In [7]:
data = data.stack().reset_index(level=1, drop=True).reset_index()

In [8]:
data.head()

beeldid       spatial                   
                      key              value
0  00035678  LOC_BUILDING  Bruxelles la nuit
1  00035678  LOC_BUILDING          la Bourse
2  00039768     LOC_PLACE    Vieux Bruxelles
3  00039607     LOC_PLACE             Laeken
4  00005577    LOC_STREET      Avenue Louise

In [9]:
data.columns = ['_'.join(col) for col in data.columns]

In [10]:
data.rename(columns={'beeldid_': 'beeldid'}, inplace=True)

In [11]:
data.head()

,beeldid,spatial_key,spatial_value
0,00035678,LOC_BUILDING,Bruxelles la nuit
1,00035678,LOC_BUILDING,la Bourse
2,00039768,LOC_PLACE,Vieux Bruxelles
3,00039607,LOC_PLACE,Laeken
4,00005577,LOC_STREET,Avenue Louise


In [12]:
#join/merge with phototheque_pallas to get the locations. 
#The csv is gzipped to get around the 100MB file weigth limitation of Github
phototeque = pd.read_csv("data/phototheque_pallas.csv.gz", compression="gzip", encoding="utf8", dtype={'beeldid': str, 'trfwnumm': str})

data = pd.merge(data, phototeque, how='left', on=['beeldid'], suffixes=['', '_x'])

In [13]:
# Pictures that contains more than a loc in the thesaurus descriptors

#data[data['loc_qid'].str.contains(",", na = False)]

In [14]:
data.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name
0,00035678,LOC_BUILDING,Bruxelles la nuit,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles
1,00035678,LOC_BUILDING,la Bourse,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles
2,00039768,LOC_PLACE,Vieux Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,L'Exposition internationale et universelle de ...,"Exposition universelle à Bruxelles, 1935. - 11...",3218,NaN,NaN,NaN,NaN,NaN,NaN
3,00039607,LOC_PLACE,Laeken,http://pallas.cegesoma.be/pls/opac/opac.search...,Course cycliste de vitesse d'amateurs du Club ...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN
4,00005577,LOC_STREET,Avenue Louise,http://pallas.cegesoma.be/pls/opac/opac.search...,"Avenue Louise, côté Bois de la Cambre, à Bruxe...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles


In [15]:
# Merge with NER_classes (spatial_keys matched with wikidata)
ner_classes = pd.read_csv("data/ner_classes.csv")

data = pd.merge(data, ner_classes, how='left', left_on = ['spatial_key'], right_on = ['ner_class'])

In [16]:
data.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class,wiki_qid,wiki_class
0,00035678,LOC_BUILDING,Bruxelles la nuit,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,LOC_BUILDING,Q41176,building
1,00035678,LOC_BUILDING,la Bourse,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,LOC_BUILDING,Q41176,building
2,00039768,LOC_PLACE,Vieux Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,L'Exposition internationale et universelle de ...,"Exposition universelle à Bruxelles, 1935. - 11...",3218,NaN,NaN,NaN,NaN,NaN,NaN,LOC_PLACE,Q56061,administrative territorial entity
3,00039607,LOC_PLACE,Laeken,http://pallas.cegesoma.be/pls/opac/opac.search...,Course cycliste de vitesse d'amateurs du Club ...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,LOC_PLACE,Q56061,administrative territorial entity
4,00005577,LOC_STREET,Avenue Louise,http://pallas.cegesoma.be/pls/opac/opac.search...,"Avenue Louise, côté Bois de la Cambre, à Bruxe...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,LOC_STREET,Q83620,thoroughfare


In [17]:
# rename the new columns merged
data.rename(columns={'wiki_qid': 'ner_class_qid', 'wiki_class':'ner_class_name'}, inplace=True)

In [18]:
#remove duplicate columns
data = data.T.drop_duplicates().T

In [19]:
data.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00035678,LOC_BUILDING,Bruxelles la nuit,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
1,00035678,LOC_BUILDING,la Bourse,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
2,00039768,LOC_PLACE,Vieux Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,L'Exposition internationale et universelle de ...,"Exposition universelle à Bruxelles, 1935. - 11...",3218,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00039607,LOC_PLACE,Laeken,http://pallas.cegesoma.be/pls/opac/opac.search...,Course cycliste de vitesse d'amateurs du Club ...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
4,00005577,LOC_STREET,Avenue Louise,http://pallas.cegesoma.be/pls/opac/opac.search...,"Avenue Louise, côté Bois de la Cambre, à Bruxe...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare


In [20]:
# remove articles in spatial_value
data['spatial_value'] = data['spatial_value'].str.replace(pat=r"\b(le|la|l\s+'|l'|les)\b", repl="", n=1, case=False)

In [21]:
#data.to_csv(r"C:\Users\ettor\Desktop\data_ugesco.csv", header=True, index=False, encoding="utf-8")

In [22]:
data.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00035678,LOC_BUILDING,Bruxelles nuit,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
1,00035678,LOC_BUILDING,Bourse,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
2,00039768,LOC_PLACE,Vieux Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,L'Exposition internationale et universelle de ...,"Exposition universelle à Bruxelles, 1935. - 11...",3218,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00039607,LOC_PLACE,Laeken,http://pallas.cegesoma.be/pls/opac/opac.search...,Course cycliste de vitesse d'amateurs du Club ...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
4,00005577,LOC_STREET,Avenue Louise,http://pallas.cegesoma.be/pls/opac/opac.search...,"Avenue Louise, côté Bois de la Cambre, à Bruxe...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare


In [23]:
#normalize spatial_value
data['spatial_value'] = data['spatial_value'].apply(lambda x: Fingerprinter(x).get_fingerprint_nonsorted())

In [24]:
#transformer les loc_place pays en loc_country
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\countries.csv", 'r', encoding="utf8") as f:
    countries = set([Fingerprinter(country).get_fingerprint_nonsorted() for country in f])

# Grab DataFrame rows where column value is present in a list
data.loc[(data['spatial_value'].isin(countries)), ['spatial_key']] = "LOC_COUNTRY"

In [25]:
data

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00035678,LOC_BUILDING,bruxelles nuit,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
1,00035678,LOC_BUILDING,bourse,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
2,00039768,LOC_PLACE,vieux bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,L'Exposition internationale et universelle de ...,"Exposition universelle à Bruxelles, 1935. - 11...",3218,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00039607,LOC_PLACE,laeken,http://pallas.cegesoma.be/pls/opac/opac.search...,Course cycliste de vitesse d'amateurs du Club ...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
4,00005577,LOC_STREET,avenue louise,http://pallas.cegesoma.be/pls/opac/opac.search...,"Avenue Louise, côté Bois de la Cambre, à Bruxe...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
5,00005577,LOC_SPACES,bois de cambre,http://pallas.cegesoma.be/pls/opac/opac.search...,"Avenue Louise, côté Bois de la Cambre, à Bruxe...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q294440,public space
6,00005577,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,"Avenue Louise, côté Bois de la Cambre, à Bruxe...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
7,00029855,LOC_PLACE,verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Américains arrivant en masse à Verviers (la vi...,"Libération de Verviers par les alliés, 9/1944....",1764,verviers (belgique)--histoire--1944-1945 (libe...,Q31,BE,Belgique,Q202954,Verviers,Q56061,administrative territorial entity
8,00029855,LOC_PLACE,verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Américains arrivant en masse à Verviers (la vi...,"Libération de Verviers par les alliés, 9/1944....",1764,verviers (belgique)--histoire--1944-1945 (libe...,Q31,BE,Belgique,Q202954,Verviers,Q56061,administrative territorial entity
9,00007696,LOC_BUILDING,palais de justice,http://pallas.cegesoma.be/pls/opac/opac.search...,[Photo A. Ruwet - Verviers] -- -- Fortins au P...,"Vie quotidienne à Verviers, 1940-1943. - 46 ph...",161934,vie quotidienne--verviers (belgique),Q31,BE,Belgique,Q202954,Verviers,Q41176,building


In [26]:
gb = data.groupby(['beeldid'])

In [27]:
gb.head() 

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00035678,LOC_BUILDING,bruxelles nuit,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
1,00035678,LOC_BUILDING,bourse,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
2,00039768,LOC_PLACE,vieux bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,L'Exposition internationale et universelle de ...,"Exposition universelle à Bruxelles, 1935. - 11...",3218,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00039607,LOC_PLACE,laeken,http://pallas.cegesoma.be/pls/opac/opac.search...,Course cycliste de vitesse d'amateurs du Club ...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
4,00005577,LOC_STREET,avenue louise,http://pallas.cegesoma.be/pls/opac/opac.search...,"Avenue Louise, côté Bois de la Cambre, à Bruxe...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
5,00005577,LOC_SPACES,bois de cambre,http://pallas.cegesoma.be/pls/opac/opac.search...,"Avenue Louise, côté Bois de la Cambre, à Bruxe...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q294440,public space
6,00005577,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,"Avenue Louise, côté Bois de la Cambre, à Bruxe...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
7,00029855,LOC_PLACE,verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Américains arrivant en masse à Verviers (la vi...,"Libération de Verviers par les alliés, 9/1944....",1764,verviers (belgique)--histoire--1944-1945 (libe...,Q31,BE,Belgique,Q202954,Verviers,Q56061,administrative territorial entity
8,00029855,LOC_PLACE,verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Américains arrivant en masse à Verviers (la vi...,"Libération de Verviers par les alliés, 9/1944....",1764,verviers (belgique)--histoire--1944-1945 (libe...,Q31,BE,Belgique,Q202954,Verviers,Q56061,administrative territorial entity
9,00007696,LOC_BUILDING,palais de justice,http://pallas.cegesoma.be/pls/opac/opac.search...,[Photo A. Ruwet - Verviers] -- -- Fortins au P...,"Vie quotidienne à Verviers, 1940-1943. - 46 ph...",161934,vie quotidienne--verviers (belgique),Q31,BE,Belgique,Q202954,Verviers,Q41176,building


In [28]:
#Sélectionner les records qui contiennent moins de 3 loc_place (attention : brussels et bruxelles = 1 en fait. D'où, deux semble un bon compromis)
df_test = gb['spatial_key'].apply(lambda x: (x=='LOC_PLACE').sum()).reset_index(name='count_places')

In [29]:
df_test

,beeldid,count_places
0,00005542,1
1,00005577,1
2,00007696,1
3,00009424,0
4,00010259,0
5,00011369,1
6,00025960,1
7,00026808,0
8,00028665,1
9,00029348,0


In [30]:
filtre = list(df_test[df_test['count_places'] < 3]['beeldid'])  
gb_filtered = data[data['beeldid'].isin(filtre)].groupby('beeldid')

In [31]:
gb_filtered.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00035678,LOC_BUILDING,bruxelles nuit,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
1,00035678,LOC_BUILDING,bourse,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles la nuit. Les lumières et les réclame...,Vues de Bruxelles : [...-1939]. - 23 photos,214,bruxelles (belgique)--photographies,Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
2,00039768,LOC_PLACE,vieux bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,L'Exposition internationale et universelle de ...,"Exposition universelle à Bruxelles, 1935. - 11...",3218,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00039607,LOC_PLACE,laeken,http://pallas.cegesoma.be/pls/opac/opac.search...,Course cycliste de vitesse d'amateurs du Club ...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
4,00005577,LOC_STREET,avenue louise,http://pallas.cegesoma.be/pls/opac/opac.search...,"Avenue Louise, côté Bois de la Cambre, à Bruxe...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
5,00005577,LOC_SPACES,bois de cambre,http://pallas.cegesoma.be/pls/opac/opac.search...,"Avenue Louise, côté Bois de la Cambre, à Bruxe...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q294440,public space
6,00005577,LOC_PLACE,bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,"Avenue Louise, côté Bois de la Cambre, à Bruxe...","Scènes de rue à Bruxelles, [1942-1944]. - 143 ...",206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
7,00029855,LOC_PLACE,verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Américains arrivant en masse à Verviers (la vi...,"Libération de Verviers par les alliés, 9/1944....",1764,verviers (belgique)--histoire--1944-1945 (libe...,Q31,BE,Belgique,Q202954,Verviers,Q56061,administrative territorial entity
8,00029855,LOC_PLACE,verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Américains arrivant en masse à Verviers (la vi...,"Libération de Verviers par les alliés, 9/1944....",1764,verviers (belgique)--histoire--1944-1945 (libe...,Q31,BE,Belgique,Q202954,Verviers,Q56061,administrative territorial entity
9,00007696,LOC_BUILDING,palais de justice,http://pallas.cegesoma.be/pls/opac/opac.search...,[Photo A. Ruwet - Verviers] -- -- Fortins au P...,"Vie quotidienne à Verviers, 1940-1943. - 46 ph...",161934,vie quotidienne--verviers (belgique),Q31,BE,Belgique,Q202954,Verviers,Q41176,building


In [32]:
#et maintenant appliquer la concatenation uniquement si le record ne contient pas plus de deux loc_place
gb = pd.DataFrame(gb_filtered['spatial_value'].apply(lambda x: "%s" % ' '.join(x))).reset_index()

In [33]:
gb.head()

,beeldid,spatial_value
0,00005542,bruxelles grand-place
1,00005577,avenue louise bois de cambre bruxelles
2,00007696,palais de justice verviers
3,00009424,eglise de beersel
4,00010259,parc duden


In [34]:
#comment concaténer loc_name maintenant, et accéder à country_name ?
#en attendant de trouver une solution élégante, récréer un dataset contenant ces infos

gb = gb.merge(data[["beeldid", "loc_name", "country_name"]])

In [35]:
gb.head()

,beeldid,spatial_value,loc_name,country_name
0,00005542,bruxelles grand-place,Bruxelles,Belgique
1,00005542,bruxelles grand-place,Bruxelles,Belgique
2,00005577,avenue louise bois de cambre bruxelles,Bruxelles,Belgique
3,00005577,avenue louise bois de cambre bruxelles,Bruxelles,Belgique
4,00005577,avenue louise bois de cambre bruxelles,Bruxelles,Belgique


In [36]:
#on splitte les noms de pays multiples (n'existent plus)
#gb = tidy_split(gb.merge(data[["beeldid", "country_name", "loc_name"]]), column="country_name", sep=",", keep=False)

In [37]:
#on enlève les doublons
gb.drop_duplicates(subset=None, keep='first', inplace=True)

In [38]:
gb

,beeldid,spatial_value,loc_name,country_name
0,00005542,bruxelles grand-place,Bruxelles,Belgique
2,00005577,avenue louise bois de cambre bruxelles,Bruxelles,Belgique
5,00007696,palais de justice verviers,Verviers,Belgique
7,00009424,eglise de beersel,Beersel,Belgique
8,00010259,parc duden,Forest,Belgique
9,00011369,jette,Jette,Belgique
10,00025960,verviers,NaN,NaN
11,00026808,region bruxelloise,Bruxelles,Belgique
12,00028665,av des champs-elysees bruxelles,NaN,Belgique
14,00029348,palais de justice,Bruxelles,Belgique


In [39]:
#on normalise les lieux et pays
gb['loc_name'] = gb['loc_name'].apply(lambda x: Fingerprinter(str(x)).get_fingerprint_nonsorted()).str.replace('nan',  '')
gb['country_name'] = gb['country_name'].apply(lambda x: Fingerprinter(str(x)).get_fingerprint_nonsorted()).str.replace('nan',  '')

In [40]:
# on concatène le tout en supprimant les doublons

gb['loc_concat'] = gb['spatial_value'].fillna('').map(
    str) + ' ' + gb['loc_name'].fillna('').map(str) + ' ' + gb['country_name'].fillna('').map(str)

gb['loc_concat'] = gb['loc_concat'].apply(
    lambda x: ' '.join(pd.unique(x.split())))

In [41]:
gb

,beeldid,spatial_value,loc_name,country_name,loc_concat
0,00005542,bruxelles grand-place,bruxelles,belgique,bruxelles grand-place belgique
2,00005577,avenue louise bois de cambre bruxelles,bruxelles,belgique,avenue louise bois de cambre bruxelles belgique
5,00007696,palais de justice verviers,verviers,belgique,palais de justice verviers belgique
7,00009424,eglise de beersel,beersel,belgique,eglise de beersel belgique
8,00010259,parc duden,forest,belgique,parc duden forest belgique
9,00011369,jette,jette,belgique,jette belgique
10,00025960,verviers,,,verviers
11,00026808,region bruxelloise,bruxelles,belgique,region bruxelloise bruxelles belgique
12,00028665,av des champs-elysees bruxelles,,belgique,av des champs-elysees bruxelles belgique
14,00029348,palais de justice,bruxelles,belgique,palais de justice bruxelles belgique


In [42]:
gb.iat[0,4]

'bruxelles grand-place belgique'

In [43]:
#appliquer get_nominatim. ATTENTION : ajouter le pays

# Nominatim bloque les requêtes répétées
requests_cache.install_cache('nominatim_cache')

s = requests.Session()

def get_nominatim(value, countrycodes=['BE',''], limit=5, lang="fr"):
    # doc : https://wiki.openstreetmap.org/wiki/Nominatim
    url = 'http://nominatim.openstreetmap.org/'
    

    params = {'q': value,
              'format': 'jsonv2',
              'addressdetails': 1,
              'limit': limit,
              'email': 'ettorerizza@mail.com',
              'polygon_kml': 0,
              'extratags': 1,
              'namedetails': 0,
              # ajouter un country_code vide simule 
              # la péréférence pour un pays de geopy
              # sans se limiter à celui-ci
              'countrycodes': countrycodes,
              'accept-language': lang
              }

    headers = {
        'User-Agent': 'Ugesco app',
        'From': 'ettorerizza@mail.com'
    }

    result = s.get(url, params=params, headers=headers, timeout=10).json()

    # Nominatim bloque les requêtes trop rapides
    sleep(1)

    return result

gb['nominatim'] = gb['loc_concat'].apply(get_nominatim)

D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=2976, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:886d:dfde:a57f:2129', 63776, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=2980, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:886d:dfde:a57f:2129', 63778, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=2932, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:886d:dfde:a57f:2129', 63780, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=2660, family=AddressFamily.AF_INET6, type=Socket

In [44]:
gb.head(10)

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim
0,00005542,bruxelles grand-place,bruxelles,belgique,bruxelles grand-place belgique,"[{'place_id': '68150353', 'licence': 'Data © O..."
2,00005577,avenue louise bois de cambre bruxelles,bruxelles,belgique,avenue louise bois de cambre bruxelles belgique,[]
5,00007696,palais de justice verviers,verviers,belgique,palais de justice verviers belgique,"[{'place_id': '82505228', 'licence': 'Data © O..."
7,00009424,eglise de beersel,beersel,belgique,eglise de beersel belgique,"[{'place_id': '160456414', 'licence': 'Data © ..."
8,00010259,parc duden,forest,belgique,parc duden forest belgique,"[{'place_id': '69018614', 'licence': 'Data © O..."
9,00011369,jette,jette,belgique,jette belgique,"[{'place_id': '179011536', 'licence': 'Data © ..."
10,00025960,verviers,,,verviers,"[{'place_id': '178612889', 'licence': 'Data © ..."
11,00026808,region bruxelloise,bruxelles,belgique,region bruxelloise bruxelles belgique,[]
12,00028665,av des champs-elysees bruxelles,,belgique,av des champs-elysees bruxelles belgique,[]
14,00029348,palais de justice,bruxelles,belgique,palais de justice bruxelles belgique,"[{'place_id': '179447494', 'licence': 'Data © ..."


In [45]:
#Comment attribuer un score de fiabilité aux résultats et choisir le meilleur s'il y en a un ?

def get_element(array, *args):
    """parse the first nominatim json."""
    for el in array:
        return dig(el, *args)
    
gb['main_loc'] = np.vectorize(get_element)(gb['nominatim'], "display_name")
gb['type_loc'] = np.vectorize(get_element)(gb['nominatim'], "type")
gb['city_loc'] = np.vectorize(get_element)(gb['nominatim'], "address", "city")
gb['country_loc'] = np.vectorize(get_element)(gb['nominatim'], "address", "country") 

In [46]:
gb 

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,main_loc,type_loc,city_loc,country_loc
0,00005542,bruxelles grand-place,bruxelles,belgique,bruxelles grand-place belgique,"[{'place_id': '68150353', 'licence': 'Data © O...","Hôtel de ville de Bruxelles, 8, Grand-Place, Q...",attraction,Ville de Bruxelles,Belgique
2,00005577,avenue louise bois de cambre bruxelles,bruxelles,belgique,avenue louise bois de cambre bruxelles belgique,[],None,None,None,None
5,00007696,palais de justice verviers,verviers,belgique,palais de justice verviers belgique,"[{'place_id': '82505228', 'licence': 'Data © O...","Palais de Justice de Verviers, Rue du Palais, ...",courthouse,None,Belgique
7,00009424,eglise de beersel,beersel,belgique,eglise de beersel belgique,"[{'place_id': '160456414', 'licence': 'Data © ...","Onze-Lieve-Vrouw, 1, Kapelaansplein, Alsemberg...",place_of_worship,None,Belgique
8,00010259,parc duden,forest,belgique,parc duden forest belgique,"[{'place_id': '69018614', 'licence': 'Data © O...","Parc Duden, Forest, Bruxelles-Capitale, 1190, ...",park,None,Belgique
9,00011369,jette,jette,belgique,jette belgique,"[{'place_id': '179011536', 'licence': 'Data © ...","Jette, Bruxelles-Capitale, 1090, Belgique",administrative,Jette,Belgique
10,00025960,verviers,,,verviers,"[{'place_id': '178612889', 'licence': 'Data © ...","Verviers, Liège, Wallonie, 4800, Belgique",administrative,Verviers,Belgique
11,00026808,region bruxelloise,bruxelles,belgique,region bruxelloise bruxelles belgique,[],None,None,None,None
12,00028665,av des champs-elysees bruxelles,,belgique,av des champs-elysees bruxelles belgique,[],None,None,None,None
14,00029348,palais de justice,bruxelles,belgique,palais de justice bruxelles belgique,"[{'place_id': '179447494', 'licence': 'Data © ...","Palais de justice, 1, Place Poelaert, Quartier...",courthouse,Ville de Bruxelles,Belgique


In [47]:
#en attendant de trouver un moyen de chosir le bon Nominatim, traitons avec Wikipedia les beeldid dont main_loc est None
import wikipedia
def get_wikipedia(value):
    wikipedia.set_lang("fr")
    if value is not None:
        try:
            return wikipedia.search(value, results=1, suggestion=False)[0]
        except IndexError:
            return None


In [48]:
gb['wikipedia_test'] = np.vectorize(get_wikipedia)(gb['loc_concat']) 

In [49]:
gb

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,main_loc,type_loc,city_loc,country_loc,wikipedia_test
0,00005542,bruxelles grand-place,bruxelles,belgique,bruxelles grand-place belgique,"[{'place_id': '68150353', 'licence': 'Data © O...","Hôtel de ville de Bruxelles, 8, Grand-Place, Q...",attraction,Ville de Bruxelles,Belgique,Grand-Place de Bruxelles
2,00005577,avenue louise bois de cambre bruxelles,bruxelles,belgique,avenue louise bois de cambre bruxelles belgique,[],None,None,None,None,Bois de la Cambre
5,00007696,palais de justice verviers,verviers,belgique,palais de justice verviers belgique,"[{'place_id': '82505228', 'licence': 'Data © O...","Palais de Justice de Verviers, Rue du Palais, ...",courthouse,None,Belgique,Belgique
7,00009424,eglise de beersel,beersel,belgique,eglise de beersel belgique,"[{'place_id': '160456414', 'licence': 'Data © ...","Onze-Lieve-Vrouw, 1, Kapelaansplein, Alsemberg...",place_of_worship,None,Belgique,Beersel
8,00010259,parc duden,forest,belgique,parc duden forest belgique,"[{'place_id': '69018614', 'licence': 'Data © O...","Parc Duden, Forest, Bruxelles-Capitale, 1190, ...",park,None,Belgique,Parc Duden
9,00011369,jette,jette,belgique,jette belgique,"[{'place_id': '179011536', 'licence': 'Data © ...","Jette, Bruxelles-Capitale, 1090, Belgique",administrative,Jette,Belgique,Jette
10,00025960,verviers,,,verviers,"[{'place_id': '178612889', 'licence': 'Data © ...","Verviers, Liège, Wallonie, 4800, Belgique",administrative,Verviers,Belgique,Verviers
11,00026808,region bruxelloise,bruxelles,belgique,region bruxelloise bruxelles belgique,[],None,None,None,None,Région de Bruxelles-Capitale
12,00028665,av des champs-elysees bruxelles,,belgique,av des champs-elysees bruxelles belgique,[],None,None,None,None,Jean-Luc Mélenchon
14,00029348,palais de justice,bruxelles,belgique,palais de justice bruxelles belgique,"[{'place_id': '179447494', 'licence': 'Data © ...","Palais de justice, 1, Place Poelaert, Quartier...",courthouse,Ville de Bruxelles,Belgique,Palais de justice de Bruxelles


In [50]:
#deuxième version de wikipedia
import requests
import functools
from ngram import NGram


def get_similar(data, target):
    G = NGram(target)
    return G.find(data)


def get_similars(data, target, threshold):
    G = NGram(target)
    return G.search(data, threshold=threshold)[0][0]


class cache(object):

    def __init__(self, fn):
        self.fn = fn
        self._cache = {}
        functools.update_wrapper(self, fn)

    def __call__(self, *args, **kwargs):
        key = str(args) + str(kwargs)
        if key in self._cache:
            ret = self._cache[key]
        else:
            ret = self._cache[key] = self.fn(*args, **kwargs)

        return ret

    def clear_cache(self):
        self._cache = {}


@cache
def gsrsearch(query, lang="FR", results=10):
    '''
    Do a Wikipedia search for `query` and 
    return a list of tuples (page title, url).
    Use ngram module to get the best match

    Keyword arguments:

    * lang : language iso code : en, fr, etc.
    * results - the maximum number of results returned

    '''

    search_params = {
        'action': 'query',
        'generator': 'search',
        'prop': 'info',
        'inprop': 'url',
        'gsrlimit': results,
        'gsrsearch': query,
        'format': 'json'
    }

    url = "https://{}.wikipedia.org/w/api.php?".format(lang.lower())

    raw_results = requests.get(url, params=search_params).json()

    try:

        search_results = [d['title']
                          for i, d in raw_results['query']['pages'].items()]
        url_results = [d['fullurl']
                       for i, d in raw_results['query']['pages'].items()]
    except KeyError:
        return None

    try:

        most_similar = get_similars(query, search_results, 0.15)
        return most_similar + " (%s)" % (url_results[search_results.index(most_similar)])

    except IndexError:

        return list(zip(search_results, url_results))

In [51]:
gb['wikipedia_improved'] = gb['loc_concat'].apply(lambda x: gsrsearch(x))

In [52]:
gb

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,main_loc,type_loc,city_loc,country_loc,wikipedia_test,wikipedia_improved
0,00005542,bruxelles grand-place,bruxelles,belgique,bruxelles grand-place belgique,"[{'place_id': '68150353', 'licence': 'Data © O...","Hôtel de ville de Bruxelles, 8, Grand-Place, Q...",attraction,Ville de Bruxelles,Belgique,Grand-Place de Bruxelles,Grand-Place de Bruxelles (https://fr.wikipedia...
2,00005577,avenue louise bois de cambre bruxelles,bruxelles,belgique,avenue louise bois de cambre bruxelles belgique,[],None,None,None,None,Bois de la Cambre,Bois de la Cambre (https://fr.wikipedia.org/wi...
5,00007696,palais de justice verviers,verviers,belgique,palais de justice verviers belgique,"[{'place_id': '82505228', 'licence': 'Data © O...","Palais de Justice de Verviers, Rue du Palais, ...",courthouse,None,Belgique,Belgique,Belgique (https://fr.wikipedia.org/wiki/Belgique)
7,00009424,eglise de beersel,beersel,belgique,eglise de beersel belgique,"[{'place_id': '160456414', 'licence': 'Data © ...","Onze-Lieve-Vrouw, 1, Kapelaansplein, Alsemberg...",place_of_worship,None,Belgique,Beersel,"[(Alsemberg, https://fr.wikipedia.org/wiki/Als..."
8,00010259,parc duden,forest,belgique,parc duden forest belgique,"[{'place_id': '69018614', 'licence': 'Data © O...","Parc Duden, Forest, Bruxelles-Capitale, 1190, ...",park,None,Belgique,Parc Duden,Parc de Forest (https://fr.wikipedia.org/wiki/...
9,00011369,jette,jette,belgique,jette belgique,"[{'place_id': '179011536', 'licence': 'Data © ...","Jette, Bruxelles-Capitale, 1090, Belgique",administrative,Jette,Belgique,Jette,Liste des ports en Belgique (https://fr.wikipe...
10,00025960,verviers,,,verviers,"[{'place_id': '178612889', 'licence': 'Data © ...","Verviers, Liège, Wallonie, 4800, Belgique",administrative,Verviers,Belgique,Verviers,Verviers (https://fr.wikipedia.org/wiki/Verviers)
11,00026808,region bruxelloise,bruxelles,belgique,region bruxelloise bruxelles belgique,[],None,None,None,None,Région de Bruxelles-Capitale,Agglomération bruxelloise (https://fr.wikipedi...
12,00028665,av des champs-elysees bruxelles,,belgique,av des champs-elysees bruxelles belgique,[],None,None,None,None,Jean-Luc Mélenchon,"[(Adolf Hitler, https://fr.wikipedia.org/wiki/..."
14,00029348,palais de justice,bruxelles,belgique,palais de justice bruxelles belgique,"[{'place_id': '179447494', 'licence': 'Data © ...","Palais de justice, 1, Place Poelaert, Quartier...",courthouse,Ville de Bruxelles,Belgique,Palais de justice de Bruxelles,Palais de justice de Bruxelles (https://fr.wik...
